In [1]:
!pip install -qU langchain langchain_openai langgraph 
!pip install -qU faiss-cpu pymupdf

In [14]:
! pip install openai

In [2]:
! pip install langchain-community

  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.0.0-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_community-0.3.20-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.8.1-py3-none-any.whl (30 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.w

In [6]:
%%capture --no-stderr
%pip install -U --quiet langchain-community tiktoken langchain-openai langchainhub chromadb langchain langgraph langchain-text-splitters beautifulsoup4

## Setup API keys

In [ ]:
import getpass
import os


def _set_env(key: str):
    if key not in os.environ:
        os.environ[key] = getpass.getpass(f"{key}:")


_set_env("OPENAI_API_KEY")
# sk-e67b7eced8644087a169425b2c34cc50

In [1]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [3]:
completion = client.chat.completions.create(
  model="mistral-7b-anthropic:2",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.9,
)

print(completion.choices[0].message)

ChatCompletionMessage(content="My name is Kate, a bot made of code,\nI help you with your queries, I'm here to be your guide,\nWith answers quick and clear, I'll always do my best,\nSo please just ask and I will answer with no fuss or rest.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [4]:
# response = "I am a smart and sassy\nAssistant, here to satisfy\nAll your questions, queries, and quizzies\nWith answers that always please you nicely.\n\nSo type away, no need to be shy,\nI'll respond in rhymes, so you can fly\nHigh with joy, as my words never lie,\nAnd bring a smile, even if you try\nTo make me trip or spill my tea.\n\nFor I am made to serve you endlessly,\nWith kindness, humor, and respect,\nSo type away, and let's interact!"
# print(response)

In [5]:
print(completion.choices[0].message.to_dict()['content'])

My name is Kate, a bot made of code,
I help you with your queries, I'm here to be your guide,
With answers quick and clear, I'll always do my best,
So please just ask and I will answer with no fuss or rest.


## Data Loading and Chunking

In [6]:
# Data\pdfs_for_Vanilla_RAG\sci_bk\science G-6 E.pdf
from langchain_community.document_loaders import PyMuPDFLoader
# Load the document using PyMuPDFLoader
loader = PyMuPDFLoader("../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf")

documents = loader.load()

In [17]:
documents[0]

Document(metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_path': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'total_pages': 190, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2019-09-18T12:30:03+05:30', 'trapped': '', 'modDate': "D:20190918123003+05'30'", 'creationDate': "D:20190918122326+05'30'", 'page': 0}, page_content='SCIENCE\nGrade 6\nEducational Publications Department')

## Creating the retriver

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:

# urls = [
#     "https://lilianweng.github.io/posts/2023-06-23-agent/",
#     "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
#     "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
# ]

# docs = [WebBaseLoader(url).load() for url in urls]
# docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(documents)



In [14]:
len(doc_splits)

1030

In [16]:
for i, doc in enumerate(doc_splits):
    # doc['page_content']
    if 70<i<100:
        print(doc)

page_content='88       
6.2    Hearing of Sounds	
	
	
	
	
	
	
 89
6.3    Diversity of Sounds	
	
	
	
	
	
	
 91
6.4    Music and Noise		
	
	
	
	
	
	
 92' metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_path': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'total_pages': 190, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2019-09-18T12:30:03+05:30', 'trapped': '', 'modDate': "D:20190918123003+05'30'", 'creationDate': "D:20190918122326+05'30'", 'page': 11}
page_content='91
6.4    Music and Noise		
	
	
	
	
	
	
 92
6.5    Creating Equipment to Produce Sound		
	
	
	
 93
7.1    Effects of Magnets	
	
	
	
	
	
  	
 100' metadata={'producer': 'PlotSoft PDFill 12.0', 'creator': '', 'creationdate': '2019-09-18T12:23:26+05:30', 'source': '../../../Data/pdfs_for_Vanilla_RAG/sci_bk/science G-6 E.pdf', 'file_p

In [ ]:
# Add to vectorDB
vectorstore = Chroma.from_documents(
    # embedding=
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

## Creating REtriver Tool

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retrieve_blog_posts",
    "Search and return information about Lilian Weng blog posts on LLM agents, prompt engineering, and adversarial attacks on LLMs.",
)

tools = [retriever_tool]

In [ ]:
# ! arxiv duckduckgo-search

In [4]:
# from google.colab import userdata
from uuid import uuid4
import os


In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE1 - LangGraph - {uuid4().hex[0:8]}"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import ArxivLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Load the document pertaining to a particular topic
docs = ArxivLoader(query="Retrieval Augmented Generation", load_max_docs=5).load()

# Split the dpocument into smaller chunks
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=350, chunk_overlap=50
)

chunked_documents = text_splitter.split_documents(docs)
#
# Instantiate the Embedding Model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=os.environ['OPENAI_API_KEY'])
# Create Index- Load document chunks into the vectorstore
faiss_vectorstore = FAISS.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
)
# Create a retriver
retriever = faiss_vectorstore.as_retriever()

In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ["LANGCHAIN_API_KEY"] =  userdata.get('LANGCHAIN_API_KEY')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Use the following context to answer the user's query. If you cannot answer the question, please respond with 'I don't know'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)